# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("./final_data/final_train_data_ver2.csv")
test_data = pd.read_csv("./final_data/final_test_data_ver2.csv")

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result_encoded',
    'WorkMode Collect Result',
    'Receip_n_suffix_3',
    'time_gap_All'
]

In [4]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [5]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [6]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [7]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [8]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [9]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 44
train_data_dam DataFrame의 칼럼 수: 25
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 16
train_data_fill2 DataFrame의 칼럼 수: 16
----test data-----
test_data DataFrame의 칼럼 수: 45
test_data_dam DataFrame의 칼럼 수: 26
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 17
test_data_fill2 DataFrame의 칼럼 수: 17


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [10]:
# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_autoclave['target'] = train_data_autoclave['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

def objectiveExtraTreesClassifier(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2500)
        , 'max_depth': trial.suggest_int('max_depth', 2, 50)
        , 'min_samples_split': trial.suggest_int('min_samples_split', 2, 12)
        , 'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 7)
        , 'criterion': trial.suggest_categorical('criterion', ['entropy'])
        , 'bootstrap': trial.suggest_categorical('bootstrap', [False])
        , 'random_state': RANDOM_STATE
    }
    
    model = ExtraTreesClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_autoclave.drop("target", axis=1),
    train_data_autoclave["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveExtraTreesClassifier(trial, x_train, y_train, x_val, y_val), n_trials=400)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


C:\Users\KimDongyoung\AppData\Local\Temp\ipykernel_43204\2791755222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_autoclave['target'] = train_data_autoclave['target'].map({'Normal': 0, 'AbNormal': 1})
[I 2024-09-28 23:49:55,140] A new study created in memory with name: no-name-b67e879d-a8bb-4dfc-891e-690975a4c6bc
[I 2024-09-28 23:50:10,712] Trial 0 finished with value: 0.1023454157782516 and parameters: {'n_estimators': 1174, 'max_depth': 34, 'min_samples_split': 6, 'min_samples_leaf': 5, 'criterion': 'entropy', 'bootstrap': False}. Best is trial 0 with value: 0.1023454157782516.
[I 2024-09-28 23:50:37,925] Trial 1 finished with value: 0.1023454157782516 and parameters: {'n_estimators': 2019, 'max_depth': 35, 'min_samples_split':

Best trial: score 0.24096385542168672, 
params {'n_estimators': 1943, 'max_depth': 34, 'min_samples_split': 6, 'min_samples_leaf': 1, 'criterion': 'entropy', 'bootstrap': False}


Best trial: score 0.24096385542168672,   
params {'n_estimators': 1943, 'max_depth': 34, 'min_samples_split': 6, 'min_samples_leaf': 1, 'criterion': 'entropy', 'bootstrap': False}

In [50]:
# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name]

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

In [51]:
train_and_evaluate_model(
    'et', train_data_autoclave,
    n_estimators = 1679, 
    max_depth = 35, 
    min_samples_split = 6, 
    min_samples_leaf = 1, 
    criterion = 'entropy', 
    bootstrap = False,
    random_state=RANDOM_STATE
)

et 모델이 train_data_autoclave 데이터로 학습한 결과:
F1 Score: 0.23034330011074197
---
Confusion Matrix:
[[7303  359]
 [ 336  104]]
---
Accuracy: 0.9142187114292767
Precision: 0.22462203023758098
Recall: 0.23636363636363636




et 모델이 train_data_autoclave 데이터로 학습한 결과

F1 Score: 0.23034330011074197

.